<h1>Population generation</h1>

In [3]:
#necessary imports
import numpy as np
import pandas as pd
import scipy.stats
from numpy import random
from scipy.stats import truncnorm

In [4]:
#możliwe generowane pola. Generacja opiera się o przecięcie tego zbioru = Ω ze zbiorem generowanym = A ⊂ Ω
populationFields = ["age", "gender", "race", "BMI", "physicalActivity", "otherIllnesses"]

from enum import Enum
class OtherIllnesses(Enum):
    hypertension = 0
    asthma = 1
    cancer = 2
    diabetes = 3
    depression = 4
    angina = 5
    myocardialInfraction = 6
    irritableBowelSyndrome = 7
    stroke = 8
    migraine = 9
#end class


In [5]:
#każda funkcja zwraca ilość == amount danych dla całej populacji
def normal(amount, spec):
    return np.random.normal(spec[0], spec[1], amount)
def choice(amount, spec):
    return np.random.choice(spec[0], amount, True, spec[1])
def minmax (amount, spec):
    X = truncnorm((spec[0] - spec[2]) / spec[3], (spec[1] - spec[2]) / spec[3], loc=spec[2], scale=spec[3])
    return X.rvs(amount)


normal(10, [52, 12.5])
#normal(10, [27.3397, 4.77216])

array([53.25366932, 45.19118926, 51.39809651, 46.0797444 , 60.59056643,
       53.45191579, 58.18788113, 53.45608665, 31.98154139, 48.62209559])

In [10]:
#przykładowe generowanie populacji z niektórymi cechami

testspec = ["age", "gender", "race", "BMI", "physicalActivity"]
testweights = [[56, 12.5],#age[mean, standard_deviation] - normal dist.
            [["f", "m"],[0.52, 0.48]],#gender[vals, weights] - weighted single choice
            [["white","black","other"],[0.66, 0.22, 0.12]],#race[vals, weights] - weighted single choice
              [12.1212, 44.8364, 27.3397, 4.77216 ], #BMI [min_val, max_val, mean, std]] -beta
              [0, 510, 127.095, 101.904]] #activityMinutes [min_val, max_val, mean, std]] -beta
            #otherIllnesses będziemy modelować z wykorzystaniem metody weighted single choice wybierając
            #ułamki wg występowania choroby w populacji
#raczej przekazanie funkcji zamiast stringów
testdistrmethods = [normal, choice, choice, minmax, minmax]
#testdistrmethods = [normal, choice, choice, normal]
testdata = [testspec, testweights, testdistrmethods]

In [11]:
def populateDatabase(amount, speclist):
    #init rng for data generation
    #rng = default_rng()
    #przecięcie zboiru generowanych cech i wszystkich możliwych
    columns = set(speclist[0]).intersection(populationFields)
    
    df = pd.DataFrame()
    for index,vals,method in zip(speclist[0],speclist[1],speclist[2]) :
        df[index] = method(amount, vals)
    
    return df
populateDatabase(15, testdata)

,age,gender,race,BMI,physicalActivity
0,61.197142,m,white,21.768750,226.827315
1,40.387360,m,white,25.092855,248.944064
2,51.328832,f,white,24.337559,49.156356
3,67.088723,f,black,31.470651,52.678270
4,67.823123,f,white,17.994798,275.777356
5,45.219470,m,other,28.304997,138.404224
6,62.555751,f,black,23.269476,213.700715
7,44.857734,m,black,36.539506,60.933521
8,58.625026,m,black,34.744398,203.972152
9,72.209995,m,white,23.122745,179.181387
